<a href="https://colab.research.google.com/github/r-u-5k/study-dev/blob/master/bigdata/5-3-type2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 심장마비 확률이 높은사람?
- 성별, 나이, 혈압, 콜레스테롤, 공복혈당, 최대 심박수 등의 컬럼이 있음
- 평가: ROC-AUC, 정확도(Accuracy), F1 을 구하시오
- target : output (1:심장마비 확률 높음, 0:심장마비 확률 낮음)
- csv파일 생성 : 수험번호.csv (예시 아래 참조)
~~~
id,output
41,0.633
28,0.123
222,0.355
~~~



## 데이터 불러오기

In [85]:
import pandas as pd

train = pd.read_csv("https://raw.githubusercontent.com/lovedlim/inf/main/p2/heart/2files/train.csv")
test =  pd.read_csv("https://raw.githubusercontent.com/lovedlim/inf/main/p2/heart/2files/test.csv")
# train = pd.read_csv("train.csv")
# test =  pd.read_csv("test.csv")

##EDA

In [86]:
train.head()

,id,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall,output
0,48,53,0,2,128,216,0,0,115,0,0.0,2,0,0,1
1,124,39,0,2,94,199,0,1,179,0,0.0,2,0,2,1
2,129,74,0,1,120,269,0,0,121,1,0.2,2,1,2,1
3,78,52,1,1,128,205,1,1,184,0,0.0,2,0,2,1
4,151,71,0,0,112,149,0,1,125,0,1.6,1,0,2,1


In [87]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 242 entries, 0 to 241
Data columns (total 15 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   id        242 non-null    int64  
 1   age       242 non-null    int64  
 2   sex       242 non-null    int64  
 3   cp        242 non-null    int64  
 4   trtbps    242 non-null    int64  
 5   chol      242 non-null    int64  
 6   fbs       242 non-null    int64  
 7   restecg   242 non-null    int64  
 8   thalachh  242 non-null    int64  
 9   exng      242 non-null    int64  
 10  oldpeak   242 non-null    float64
 11  slp       242 non-null    int64  
 12  caa       242 non-null    int64  
 13  thall     242 non-null    int64  
 14  output    242 non-null    int64  
dtypes: float64(1), int64(14)
memory usage: 28.5 KB


In [88]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61 entries, 0 to 60
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   id        61 non-null     int64  
 1   age       61 non-null     int64  
 2   sex       61 non-null     int64  
 3   cp        61 non-null     int64  
 4   trtbps    61 non-null     int64  
 5   chol      61 non-null     int64  
 6   fbs       61 non-null     int64  
 7   restecg   61 non-null     int64  
 8   thalachh  61 non-null     int64  
 9   exng      61 non-null     int64  
 10  oldpeak   61 non-null     float64
 11  slp       61 non-null     int64  
 12  caa       61 non-null     int64  
 13  thall     61 non-null     int64  
dtypes: float64(1), int64(13)
memory usage: 6.8 KB


In [89]:
train.isnull().sum()

,0
id,0
age,0
sex,0
cp,0
trtbps,0
chol,0
fbs,0
restecg,0
thalachh,0
exng,0


In [90]:
train.describe()

,id,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall,output
count,242.000000,242.000000,242.000000,242.000000,242.000000,242.000000,242.000000,242.00000,242.000000,242.000000,242.000000,242.000000,242.000000,242.000000,242.000000
mean,154.706612,54.516529,0.673554,0.913223,131.070248,246.933884,0.119835,0.53719,149.590909,0.347107,1.019835,1.380165,0.785124,2.347107,0.520661
std,86.328576,9.021513,0.469885,1.020897,17.716978,52.236414,0.325441,0.53183,21.763201,0.477037,1.182137,0.621285,1.052218,0.607160,0.500608
min,0.000000,34.000000,0.000000,0.000000,94.000000,126.000000,0.000000,0.00000,88.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,81.250000,47.250000,0.000000,0.000000,120.000000,212.000000,0.000000,0.00000,133.250000,0.000000,0.000000,1.000000,0.000000,2.000000,0.000000
50%,158.500000,55.000000,1.000000,1.000000,130.000000,243.000000,0.000000,1.00000,152.000000,0.000000,0.600000,1.000000,0.000000,2.000000,1.000000
75%,228.750000,61.000000,1.000000,2.000000,140.000000,274.000000,0.000000,1.00000,165.000000,1.000000,1.600000,2.000000,1.000000,3.000000,1.000000
max,302.000000,77.000000,1.000000,3.000000,200.000000,564.000000,1.000000,2.00000,195.000000,1.000000,6.200000,2.000000,4.000000,3.000000,1.000000


In [91]:
train['output'].describe()

,output
count,242.000000
mean,0.520661
std,0.500608
min,0.000000
25%,0.000000
50%,1.000000
75%,1.000000
max,1.000000


In [92]:
train['output'].value_counts()

,count
output,
1,126
0,116


## 데이터 전처리 & 피처엔지니어링

In [93]:
train = train.drop('id', axis=1)
test_id = test.pop('id')
# train = pd.get_dummies(train)
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 242 entries, 0 to 241
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       242 non-null    int64  
 1   sex       242 non-null    int64  
 2   cp        242 non-null    int64  
 3   trtbps    242 non-null    int64  
 4   chol      242 non-null    int64  
 5   fbs       242 non-null    int64  
 6   restecg   242 non-null    int64  
 7   thalachh  242 non-null    int64  
 8   exng      242 non-null    int64  
 9   oldpeak   242 non-null    float64
 10  slp       242 non-null    int64  
 11  caa       242 non-null    int64  
 12  thall     242 non-null    int64  
 13  output    242 non-null    int64  
dtypes: float64(1), int64(13)
memory usage: 26.6 KB


## 검증 데이터 분리

In [94]:
target = train.pop('output')

In [95]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(train, target, test_size=0.2, random_state=0)

## 모델 & 평가

In [96]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(random_state=0)
model.fit(X_train, y_train)
pred = model.predict(X_val)
pred_proba = model.predict_proba(X_val)

In [97]:
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score

print(roc_auc_score(y_val, pred_proba[:, 1]))
print(accuracy_score(y_val, pred))
print(f1_score(y_val, pred))

0.9297658862876255
0.8571428571428571
0.8679245283018868


In [98]:
model.classes_

array([0, 1])

## 예측 및 csv 제출

In [101]:
pred_test = model.predict_proba(test)
pred_test[:10]

array([[0.08, 0.92],
       [0.32, 0.68],
       [0.55, 0.45],
       [0.56, 0.44],
       [0.01, 0.99],
       [0.9 , 0.1 ],
       [0.78, 0.22],
       [0.04, 0.96],
       [0.79, 0.21],
       [0.52, 0.48]])

In [102]:
result = pd.DataFrame({'id': test_id, 'output': pred_test[:, 1]})

In [106]:
result.to_csv('result.csv', index=False)

In [107]:
pd.read_csv('result.csv')

,id,output
0,41,0.92
1,28,0.68
2,222,0.45
3,106,0.44
4,108,0.99
...,...,...
56,6,0.85
57,269,0.01
58,2,0.97
59,14,0.78


##정리

In [99]:
# y_test = pd.read_csv("https://raw.githubusercontent.com/lovedlim/inf/main/p2/heart/2files/y_test.csv")
# print(roc_auc_score(y_test, pred_proba[:,1]))